[Home](../../README.md)

### Feature Engineering

This Jupyter Notepad is a selection of data engineering processes you can apply to your data before model training to maximise the performance of your machine learning model. For this demonstration we will engineer new or improved features for the diabetes data you previously wrangled.

#### Feature Engineering Process
- Deriving new variables from existing ones
    - Encoding categorical features
    - Calculating new features from existing features
- Combining features/feature interactions
- Identifying the most relevant features for the model
- Transforming Features
  - [Dividing Data into categories](https://web.ma.utexas.edu/users/mks/statmistakes/dividingcontinuousintocategories.html)
  - Mathematical transformations (for example logarithmic transformations). Logarithmic transformations are a powerful tool in the world of statistical analysis. They are often used to transform data that exhibit skewness or other irregularities, making it easier to analyze, visualize, and interpret the results.
- Creating Domain-Specific Features that incorporating knowledge from the specific domain to create features that capture important characteristics of the data.

#### Load the required dependencies

In [1]:
# Import frameworks
import pandas as pd

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [2]:
data_frame = pd.read_csv("2.2.1.wrangled_data.csv")

####  Deriving new variables from existing ones

##### Encoding categorical variables

Data Encoding converts textual data into numerical format, so that it can be used as input for algorithms to process. The reason for encoding is that most machine learning algorithms work with numbers and not with text or categorical variables.

To encode the 'SEX' column you will assigning a number value to the gender. Because the data set only provides 2 values we will use -1 and 1.

In [3]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: -1 if gender.lower() == 'male' else 1 if gender.lower() == 'female' else None)
print(data_frame['SEX'].head())

0    1
1    1
2   -1
3   -1
4   -1
Name: SEX, dtype: int64


##### Calculating Age

In the context of medical diagnosis of a lifestyle disease a persons date of birth has limited influence on the target. However, their age is highly relevant. So we will convert two dates into a age. You could consider further encoding this into age brackets or scalling the data, don't forget to use [2.1.2.data.records.md](../2.1.Data_Wrangling/2.1.2.data.records.md) to record any scalling or encoding.

In [4]:
# Convert the 'DoB' and 'DoTest' columns to datetime
data_frame['DoB'] = pd.to_datetime(data_frame['DoB'], format='%d/%m/%Y')
data_frame['DoT'] = pd.to_datetime(data_frame['DoT'], format='%d/%m/%Y')

# Calculate the year difference
data_frame['Age'] = ((data_frame['DoT'] - data_frame['DoB']).dt.days  / 365.25).round()

# Print the result
print(data_frame[['DoB', 'DoT', 'Age']])

           DoB        DoT   Age
0   2001-06-17 2024-10-09  23.0
1   1995-06-10 2024-08-16  29.0
2   1981-03-11 2024-03-08  43.0
3   2002-05-15 2024-05-06  22.0
4   2001-06-21 2024-07-20  23.0
..         ...        ...   ...
434 1980-04-28 2024-04-03  44.0
435 1987-06-13 2024-06-09  37.0
436 1988-09-02 2024-12-24  36.0
437 1989-08-19 2024-08-04  35.0
438 1978-02-06 2024-02-20  46.0

[439 rows x 3 columns]


#### Combining features/feature interactions

While individual features can be powerful predictors, their interactions often carry even more information. Feature interaction engineering is the process of creating new features that represent the interaction between two or more features.

In this, case some domain knowledge and data analysis have informed you that the BMI and AGE are risk multipliers (the greater the age and the greater the BMI the greater the feature). From this we can  risk value based on the feature interactions.

In [5]:
# Calculate the year difference and round to an integer
data_frame['Age'] = ((data_frame['DoT'] - data_frame['DoB']).dt.days / 365.25).round().astype(int)

# Create the 'Risk' column
data_frame['Risk'] = data_frame['BMI'] * data_frame['Age']

# Calculate the percentage of the maximum risk
data_frame['Risk%'] = (data_frame['Risk'] / data_frame['Risk'].max()).round(2)

# Print the result
print(data_frame[['Age', 'BMI', 'Risk%']])

     Age   BMI  Risk%
0     23  18.0   0.16
1     29  18.1   0.21
2     43  18.5   0.31
3     22  18.6   0.16
4     23  18.8   0.17
..   ...   ...    ...
434   44  38.2   0.66
435   37  38.3   0.56
436   36  39.1   0.55
437   35  41.3   0.57
438   46  42.2   0.76

[439 rows x 3 columns]


#### Transforming Features

Filtering is like applying the where clause in a database. It is widely used and can help when you need to work on a specific subset of your data. For our use case, let us filter the data to only include rows where the 'SEX' is 'Male'. There is no method call for this, we can just use conditional indexing to fulfil our purpose.

In this, case some domain knowledge and data analysis have informed you that there is 'bimodality' in the data and males and females have a different trends. 

In [6]:
# Filter the data to -1 only
data_frame = data_frame[data_frame['SEX'] == -1]

# Print the result
print(data_frame[['Age', 'SEX', 'Target']])

     Age  SEX  Target
2     43   -1    90.0
3     22   -1   101.0
4     23   -1    85.0
5     26   -1    51.0
6     33   -1    72.0
..   ...  ...     ...
428   57   -1   270.0
429   55   -1   258.0
431   69   -1   237.0
435   37   -1   259.0
437   35   -1   346.0

[234 rows x 3 columns]


#### Creating Domain-Specific Features

Domain knowledge is about understanding the domain or subject area of the data. In This case the domain is 'health' and more specifically   'Epidemiology' which is the study of how often diseases occur in different groups of people and why.

The column called '1st Degree Relatives' is a domain specific feature as is records the number of family members in the individuals direct bloodline who have developed type 2 adult onset diabetes. Domain specific knowledge, is that Family history of disease in first degree relatives is a major risk factor, especially for premature events.

First we will convert convert the FDR value to a risk percentage, because the risk can never be 0 (will never happen) or 100% (will definitely happen) we will scale the result between 0.15 and 0.85.

In [7]:
# Calculate the family history risk
data_frame['FHRisk'] = (data_frame['FDR'] / data_frame['FDR'].max())

# Scale the result between 0.15 and 0.85
min_val = 0.15
max_val = 0.85
data_frame['FHRisk'] = (((data_frame['FHRisk'] - data_frame['FHRisk'].min()) / (data_frame['FHRisk'].max() - data_frame['FHRisk'].min())) * (max_val - min_val) + min_val).round(2)

# Print the result
print(data_frame[['Age', 'FDR', 'FHRisk']])

     Age  FDR  FHRisk
2     43    2    0.62
3     22    2    0.62
4     23    2    0.62
5     26    2    0.62
6     33    0    0.15
..   ...  ...     ...
428   57    1    0.38
429   55    2    0.62
431   69    2    0.62
435   37    0    0.15
437   35    3    0.85

[234 rows x 3 columns]


Then to make it even more meaningful, we will combine it with the `Risk` feature we engineered using the `AGE` and `BMI` features to create a combined risk 'interaction feature' that captures real-world relationships between the features.

Again we will scale the result between 0.15 and 0.85.

In [8]:
data_frame['CombRisk'] = (data_frame['FHRisk'] * data_frame['Risk%']).round(2)

min_val = 0.15
max_val = 0.85
data_frame['CombRisk'] = (((data_frame['CombRisk'] - data_frame['CombRisk'].min()) / (data_frame['CombRisk'].max() - data_frame['CombRisk'].min())) * (max_val - min_val) + min_val).round(2)

# Print the result
print(data_frame[['Age', 'Risk%', 'FHRisk', 'CombRisk']])

     Age  Risk%  FHRisk  CombRisk
2     43   0.31    0.62      0.35
3     22   0.16    0.62      0.24
4     23   0.17    0.62      0.26
5     26   0.19    0.62      0.27
6     33   0.24    0.15      0.17
..   ...    ...     ...       ...
428   57   0.81    0.38      0.49
429   55   0.79    0.62      0.70
431   69   1.00    0.62      0.85
435   37   0.56    0.15      0.22
437   35   0.57    0.85      0.69

[234 rows x 4 columns]


#### Save the wrangled and engineered data to CSV

In [9]:
data_frame.to_csv('../2.3.Model_Training/2.3.1.model_ready_data.csv', index=False)